In [1]:
#!/usr/bin/env python3
import numpy as np
# import PyQt5  # make sure pyqtgraph imports Qt5
# import pyqtgraph
# from PyQt5 import QtCore, QtGui
# from PyQt5.QtWidgets import (
#     QWidget, QHBoxLayout, QVBoxLayout, QTabWidget, QCheckBox
# )
# from PyQt5.QtCore import Qt
# from artiq.applets.simple import TitleApplet
# from scipy.optimize import curve_fit
import pyvisa as visa
from time import sleep
import sys

    

def ramp_mcp_voltage(TODO, V1=2400, V2=2200, V3=200, sleeptime=0.5, print_log=False): 
    rm = visa.ResourceManager()
    instruments = rm.list_resources()
    # instruments
    usb = list(filter(lambda x: 'USB' in x, instruments))
    if len(usb) != 1:
        print('Bad instrument list', instruments)
        sys.exit(-1)
    odp = rm.open_resource(usb[0])
    odp.write("INST:NSEL 1")
    odp.write("VOLT:PROT 10.1")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    odp.write("INST:NSEL 2")
    odp.write("VOLT:PROT 10.1")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    odp.write("INST:NSEL 3")
    odp.write("VOLT:PROT 5")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    Vstep01 = 5*50/5000
    Vstep02 = 5*50/2500 * 1.8
    Vstep03 = 5*50/2500
    V1g0 = 10*V1/5000
    V2g0 = 10*V2/2500
    V3g0 = 10*V3/1250

    ON = "Turn on"
    OFF = "Turn off"

    if TODO == ON:
        odp.write("INST:NSEL 1")
        vinit1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 2")
        vinit2 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 3")
        vinit3 = float(odp.query("MEAS:VOLT?"))
        while 1:
            odp.write("INST:NSEL 1")
            vcurr1 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            odp.write("INST:NSEL 2")
            vcurr2 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            odp.write("INST:NSEL 3")
            vcurr3 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            if vcurr1 > V1g0+0.1:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        sleep(sleeptime)
            elif vcurr1 < V1g0-0.1:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        sleep(sleeptime)       
            else:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        break
            if print_log: 
                print(vcurr1, vcurr2, vcurr3)
        odp.write("INST:NSEL 1")
        odp.write("VOLT "+str(V1g0))
        odp.write("INST:NSEL 2")
        odp.write("VOLT "+str(V2g0))
        odp.write("INST:NSEL 3")
        odp.write("VOLT "+str(V3g0))
        print("Finish")
        if print_log: 
            print(vcurr1, vcurr2, vcurr3)

    elif TODO == OFF:
        odp.write("INST:NSEL 1")
        vinit1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 2")
        vinit2 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 3")
        vinit3 = float(odp.query("MEAS:VOLT?"))
        while 1:
            odp.write("INST:NSEL 1")
            vcurr1 = float(odp.query("MEAS:VOLT?"))
            sleep(0.1)
            odp.write("INST:NSEL 2")
            vcurr2 = float(odp.query("MEAS:VOLT?"))
            odp.write("INST:NSEL 3")
            vcurr3 = float(odp.query("MEAS:VOLT?"))
            sleep(0.1)
            if vcurr1 > 0.11:
                if vcurr2 > 0.11:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)                    
                else:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        sleep(sleeptime)                    
                    
            else:
                if vcurr2 > 0.11:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)                    
                else:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        break
            if print_log:        
                print(vcurr1, vcurr2, vcurr3)
        odp.write("INST:NSEL 1")
        odp.write("VOLT 0.01")
        odp.write("INST:NSEL 2")
        odp.write("VOLT 0.01")                    
        odp.write("INST:NSEL 3")
        odp.write("VOLT 0.01")                    
        
        
    else:
        print("Check your spelling.")

In [4]:
rm = visa.ResourceManager('@py')
instruments = rm.list_resources()
usb = list(filter(lambda x: 'USB' in x, instruments))
usb, instruments

([],
 ('ASRL/dev/ttyS4::INSTR',
  'ASRL/dev/ttyS31::INSTR',
  'ASRL/dev/ttyS30::INSTR',
  'ASRL/dev/ttyS29::INSTR',
  'ASRL/dev/ttyS28::INSTR',
  'ASRL/dev/ttyS27::INSTR',
  'ASRL/dev/ttyS26::INSTR',
  'ASRL/dev/ttyS25::INSTR',
  'ASRL/dev/ttyS24::INSTR',
  'ASRL/dev/ttyS23::INSTR',
  'ASRL/dev/ttyS22::INSTR',
  'ASRL/dev/ttyS21::INSTR',
  'ASRL/dev/ttyS20::INSTR',
  'ASRL/dev/ttyS19::INSTR',
  'ASRL/dev/ttyS18::INSTR',
  'ASRL/dev/ttyS17::INSTR',
  'ASRL/dev/ttyS16::INSTR',
  'ASRL/dev/ttyS15::INSTR',
  'ASRL/dev/ttyS14::INSTR',
  'ASRL/dev/ttyS13::INSTR',
  'ASRL/dev/ttyS12::INSTR',
  'ASRL/dev/ttyS11::INSTR',
  'ASRL/dev/ttyS10::INSTR',
  'ASRL/dev/ttyS9::INSTR',
  'ASRL/dev/ttyS8::INSTR',
  'ASRL/dev/ttyS7::INSTR',
  'ASRL/dev/ttyS6::INSTR',
  'ASRL/dev/ttyS5::INSTR',
  'ASRL/dev/ttyS3::INSTR',
  'ASRL/dev/ttyS2::INSTR',
  'ASRL/dev/ttyS1::INSTR',
  'ASRL/dev/ttyS0::INSTR',
  'TCPIP::192.168.169.115::INSTR',
  'TCPIP::192.168.169.182::INSTR'))

In [6]:
ramp_mcp_voltage("Turn on", V1=2391, V2=2205, sleeptime=2)

Finish


In [2]:
ramp_mcp_voltage("Turn off", V1=2391, V2=2203)

Bad instrument list ('ASRL/dev/ttyS4::INSTR', 'ASRL/dev/ttyS31::INSTR', 'ASRL/dev/ttyS30::INSTR', 'ASRL/dev/ttyS29::INSTR', 'ASRL/dev/ttyS28::INSTR', 'ASRL/dev/ttyS27::INSTR', 'ASRL/dev/ttyS26::INSTR', 'ASRL/dev/ttyS25::INSTR', 'ASRL/dev/ttyS24::INSTR', 'ASRL/dev/ttyS23::INSTR', 'ASRL/dev/ttyS22::INSTR', 'ASRL/dev/ttyS21::INSTR', 'ASRL/dev/ttyS20::INSTR', 'ASRL/dev/ttyS19::INSTR', 'ASRL/dev/ttyS18::INSTR', 'ASRL/dev/ttyS17::INSTR', 'ASRL/dev/ttyS16::INSTR', 'ASRL/dev/ttyS15::INSTR', 'ASRL/dev/ttyS14::INSTR', 'ASRL/dev/ttyS13::INSTR', 'ASRL/dev/ttyS12::INSTR', 'ASRL/dev/ttyS11::INSTR', 'ASRL/dev/ttyS10::INSTR', 'ASRL/dev/ttyS9::INSTR', 'ASRL/dev/ttyS8::INSTR', 'ASRL/dev/ttyS7::INSTR', 'ASRL/dev/ttyS6::INSTR', 'ASRL/dev/ttyS5::INSTR', 'ASRL/dev/ttyS3::INSTR', 'ASRL/dev/ttyS2::INSTR', 'ASRL/dev/ttyS1::INSTR', 'ASRL/dev/ttyS0::INSTR', 'TCPIP::192.168.169.115::INSTR', 'TCPIP::192.168.169.182::INSTR')


/home/electron/miniconda3/envs/artiq/lib/python3.8/site-packages/pyvisa_py/tcpip.py:122: UserWarning: TCPIP::hislip resource discovery requires the zeroconf package to be installed... try 'pip install zeroconf'
  warnings.warn(


SystemExit: -1

/home/electron/miniconda3/envs/artiq/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
V01 = -9
V02 = 3
ramp_mcp_voltage("Turn on", V1=2250+V01, V2=2150+V02, V3=150, sleeptime=2)

Finish


In [18]:
for i in instruments:
    try:
        odp = rm.open_resource(i)
        odp.write("INST:NSEL 1")
        print('>>>', i)
    except:
        print(i, 'failed')
#odp.write("VOLT:PROT 10.1")

ASRL/dev/ttyS4::INSTR failed
ASRL/dev/ttyS31::INSTR failed
ASRL/dev/ttyS30::INSTR failed
ASRL/dev/ttyS29::INSTR failed
ASRL/dev/ttyS28::INSTR failed
ASRL/dev/ttyS27::INSTR failed
ASRL/dev/ttyS26::INSTR failed
ASRL/dev/ttyS25::INSTR failed
ASRL/dev/ttyS24::INSTR failed
ASRL/dev/ttyS23::INSTR failed
ASRL/dev/ttyS22::INSTR failed
ASRL/dev/ttyS21::INSTR failed
ASRL/dev/ttyS20::INSTR failed
ASRL/dev/ttyS19::INSTR failed
ASRL/dev/ttyS18::INSTR failed
ASRL/dev/ttyS17::INSTR failed
ASRL/dev/ttyS16::INSTR failed
ASRL/dev/ttyS15::INSTR failed
ASRL/dev/ttyS14::INSTR failed
ASRL/dev/ttyS13::INSTR failed
ASRL/dev/ttyS12::INSTR failed
ASRL/dev/ttyS11::INSTR failed
ASRL/dev/ttyS10::INSTR failed
ASRL/dev/ttyS9::INSTR failed
ASRL/dev/ttyS8::INSTR failed
ASRL/dev/ttyS7::INSTR failed
ASRL/dev/ttyS6::INSTR failed
ASRL/dev/ttyS5::INSTR failed
ASRL/dev/ttyS3::INSTR failed
ASRL/dev/ttyS2::INSTR failed
ASRL/dev/ttyS1::INSTR failed
ASRL/dev/ttyS0::INSTR failed
>>> TCPIP::192.168.169.115::INSTR
>>> TCPIP::192.1